In [5]:
# Step 1: Install dependencies
!pip install -q transformers datasets peft accelerate bitsandbytes

# Step 2: Imports
import os
import json
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)

# Step 3: Sample Q&A dataset (5 examples)
qa_pairs = [
    {
        "question": "What is the legal age of marriage for a Muslim girl in Pakistan?",
        "answer": "The legal age of marriage for a Muslim girl in Pakistan is 16 years under the Child Marriage Restraint Act. However, there are proposals to raise it to 18 years."
    },
    {
        "question": "Can a girl marry without her wali (guardian) in Pakistan?",
        "answer": "Yes, under Hanafi jurisprudence and Pakistani law, an adult and sane girl can contract marriage without her wali."
    },
    {
        "question": "Is registration of nikah compulsory in Pakistan?",
        "answer": "Yes, under the Muslim Family Laws Ordinance 1961, registration of nikah is mandatory for it to be legally recognized."
    },
    {
        "question": "What is a Nikahnama?",
        "answer": "Nikahnama is the official marriage contract form in Pakistan, documenting the marriage and conditions agreed by both parties."
    },
    {
        "question": "Can conditions be included in the Nikahnama?",
        "answer": "Yes, any lawful conditions mutually agreed upon by the bride and groom can be included in the Nikahnama."
    },
    {
        "question": "Is a verbal nikah valid in Islam?",
        "answer": "Yes, a verbal nikah is valid in Islam if all requirements such as offer, acceptance, witnesses, and mahr are fulfilled."
    },
    {
        "question": "Is consent of the bride necessary in a nikah?",
        "answer": "Yes, a valid nikah cannot be performed without the free and informed consent of the bride."
    },
    {
        "question": "What is Mahr (Dower)?",
        "answer": "Mahr is a mandatory payment by the husband to the wife at the time of marriage, as an Islamic obligation."
    },
    {
        "question": "Can the wife waive her mahr?",
        "answer": "Yes, a wife can waive her mahr after marriage, provided the decision is made voluntarily and without coercion."
    },
    {
        "question": "What are the types of mahr?",
        "answer": "Mahr can be prompt (mu'ajjal) or deferred (mu’akhkhar), meaning it is either paid immediately or at a later agreed time."
    },
    {
        "question": "Can a man contract a second marriage without the first wife's permission?",
        "answer": "No, under Pakistani law, a man must obtain written permission from the Arbitration Council and the first wife before contracting a second marriage."
    },
    {
        "question": "What is the punishment for contracting a second marriage without permission?",
        "answer": "The husband may face up to one year of imprisonment or a fine up to Rs. 500,000, or both, for violating the law."
    },
    {
        "question": "Is muta (temporary marriage) recognized in Pakistan?",
        "answer": "No, muta is not legally recognized in Pakistan, which follows the Sunni Hanafi school of thought."
    },
    {
        "question": "Can a Christian woman convert to Islam and marry a Muslim man in Pakistan?",
        "answer": "Yes, if the conversion is genuine and not solely for the purpose of marriage, she may marry a Muslim man."
    },
    {
        "question": "Is registration of second marriage compulsory?",
        "answer": "Yes, just like the first marriage, all subsequent marriages must also be registered legally."
    },
    {
        "question": "What is Talaq?",
        "answer": "Talaq is the Islamic term for divorce initiated by the husband. It can be oral or written, but legal procedures must be followed for it to be effective in Pakistan."
    },
    {
        "question": "What is the procedure of talaq in Pakistan?",
        "answer": "The husband must send a written notice to the Union Council, which initiates a 90-day reconciliation period. If reconciliation fails, divorce becomes effective after this period."
    },
    {
        "question": "Is oral talaq valid in Pakistan?",
        "answer": "Religiously it may be valid, but legally it is not recognized unless a written notice is given to the Union Council."
    },
    {
        "question": "What is the iddah period after talaq?",
        "answer": "The iddah period is three menstrual cycles, or in the case of a pregnant woman, until childbirth."
    },
    {
        "question": "Can a woman initiate divorce in Pakistan?",
        "answer": "Yes, a woman can seek divorce through Khula, judicial divorce, or if delegated talaq (Talaq-e-Tafweez) is provided in the Nikahnama."
    },
    {
        "question": "What is Khula?",
        "answer": "Khula is a woman’s right to seek divorce by returning her mahr or offering compensation, usually through the Family Court."
    },
    {
        "question": "What is delegated talaq (Talaq-e-Tafweez)?",
        "answer": "It is the right given to the wife in the Nikahnama to pronounce talaq on behalf of herself, if agreed upon by the husband at marriage."
    },
    {
        "question": "What happens if a man gives triple talaq at once?",
        "answer": "Triple talaq is discouraged and counts as one revocable talaq under Pakistani legal interpretation, but legal procedures must still be followed."
    },
    {
        "question": "Can a woman remarry the same man after talaq?",
        "answer": "Yes, if talaq was revocable or if it was a single talaq. After triple talaq, remarriage requires halala (intervening marriage with another man)."
    },
    {
        "question": "What is halala?",
        "answer": "Halala is the requirement that after triple talaq, the woman must marry and be divorced by another man before she can remarry her former husband."
    }
]
qa_pairs += [
    {
        "question": "Is court approval necessary for khula in Pakistan?",
        "answer": "Yes, khula must be granted by a Family Court in Pakistan after the wife files a suit and shows valid reasons."
    },
    {
        "question": "Can a woman file for judicial divorce in Pakistan?",
        "answer": "Yes, under the Dissolution of Muslim Marriages Act 1939, a woman can seek judicial divorce on various grounds like cruelty, desertion, or impotence."
    },
    {
        "question": "What are valid grounds for judicial divorce?",
        "answer": "Grounds include husband's cruelty, desertion for two years, failure to maintain for two years, imprisonment, impotence, or failure to perform marital obligations."
    },
    {
        "question": "Is maintenance obligatory after divorce?",
        "answer": "Yes, a husband is obligated to provide maintenance during the iddah period. After that, it depends on the circumstances and court orders."
    },
    {
        "question": "What is the law on child custody after divorce?",
        "answer": "Custody is usually granted to the mother, especially for young children, but the court decides based on the best interest of the child."
    },
    {
        "question": "Can the father claim custody of children?",
        "answer": "Yes, the father can apply for custody, particularly after the child reaches a certain age or if the mother is deemed unfit."
    },
    {
        "question": "What is guardianship (wilayah) under Islamic law?",
        "answer": "Guardianship refers to the legal responsibility over a child's person or property. Usually, the father retains guardianship even if the mother has custody."
    },
    {
        "question": "What is the difference between custody and guardianship?",
        "answer": "Custody is the right to physical care of the child, while guardianship refers to legal decision-making authority and financial responsibilities."
    },
    {
        "question": "Can a mother remarry and still retain custody?",
        "answer": "Courts may transfer custody to the father or another guardian if the mother remarries, depending on the child's welfare."
    },
    {
        "question": "What is maintenance (nafqah) under Islamic law?",
        "answer": "Maintenance refers to the husband’s obligation to provide food, clothing, shelter, and medical care to his wife and children."
    },
    {
        "question": "Can a woman claim past maintenance?",
        "answer": "Yes, she may claim arrears of maintenance for a reasonable past period, subject to court discretion."
    },
    {
        "question": "Is the father's consent required for a child's travel abroad after divorce?",
        "answer": "Yes, generally the non-custodial parent's consent is required for international travel, especially when there’s a court decree involved."
    },
    {
        "question": "Can the mother apply for child maintenance separately?",
        "answer": "Yes, the mother can file a separate suit in Family Court for child maintenance from the father."
    },
    {
        "question": "What is the role of Union Councils in family disputes?",
        "answer": "Union Councils handle reconciliation in talaq cases, issue marriage/divorce certificates, and supervise arbitration proceedings."
    },
    {
        "question": "Are family court proceedings public in Pakistan?",
        "answer": "No, Family Court proceedings are generally held in-camera to protect privacy and sensitive issues."
    },
    {
        "question": "Can a husband revoke talaq during iddah?",
        "answer": "Yes, if it is the first or second talaq, the husband may revoke it during the iddah period through verbal or written declaration or resuming marital relations."
    },
    {
        "question": "Can talaq be given through SMS or WhatsApp?",
        "answer": "While some courts may accept electronic communication as evidence, legal procedures such as notice to the Union Council are still mandatory."
    },
    {
        "question": "Is forced marriage allowed under Islamic law?",
        "answer": "No, Islam strictly prohibits forced marriages. Consent of both parties is a fundamental requirement."
    },
    {
        "question": "What is the punishment for forced marriage in Pakistan?",
        "answer": "Forced marriage is a criminal offense under the Prevention of Anti-Women Practices Act, punishable with imprisonment and fine."
    },
    {
        "question": "Can a minor girl be married under Islamic law?",
        "answer": "Marriage of a minor is not valid unless she reaches puberty and gives free consent. Pakistani law also sets a minimum legal age."
    },
    {
        "question": "What is the role of arbitration councils in second marriages?",
        "answer": "Arbitration Councils evaluate applications for second marriages, ensuring that valid reasons and first wife’s consent are provided."
    },
    {
        "question": "What documents are required for Nikah registration?",
        "answer": "Documents usually include CNICs of bride, groom, and witnesses, photographs, and filled Nikahnama with signatures."
    },
    {
        "question": "Can a woman be a nikah registrar in Pakistan?",
        "answer": "Currently, Nikah registrars are typically male, but legally there is no restriction preventing women from being appointed."
    },
    {
        "question": "What is the legal status of an unregistered nikah?",
        "answer": "An unregistered nikah may still be valid religiously but is not recognized legally without proper documentation."
    },
    {
        "question": "Can a divorced woman claim inheritance from her former husband?",
        "answer": "No, a divorced woman is not entitled to inherit from her former husband unless he passes away during her iddah."
    }
]
qa_pairs += [
    {
        "question": "What happens if a husband dies during the wife’s iddah after talaq?",
        "answer": "If the husband dies during the iddah period following a revocable talaq, the wife is entitled to inheritance as a widow."
    },
    {
        "question": "What is the iddah period after the death of a husband?",
        "answer": "The iddah period after a husband's death is four months and ten days, or until delivery if the wife is pregnant."
    },
    {
        "question": "Can a woman marry during her iddah period?",
        "answer": "No, a woman cannot marry another man until her iddah period is completed."
    },
    {
        "question": "What is the punishment for child marriage in Pakistan?",
        "answer": "Child marriage is punishable with imprisonment up to three years and a fine under the Child Marriage Restraint Act."
    },
    {
        "question": "Can both parties add conditions in the Nikahnama?",
        "answer": "Yes, both the bride and groom can mutually agree to add lawful conditions in the Nikahnama."
    },
    {
        "question": "What is the role of witnesses in nikah?",
        "answer": "Two adult Muslim witnesses are required for the validity of a nikah under Islamic and Pakistani law."
    },
    {
        "question": "What if the Nikahnama is lost or not available?",
        "answer": "A duplicate can be obtained from the relevant Union Council or Nikah registrar’s office where the marriage was registered."
    },
    {
        "question": "What is an ex-parte decree in family law?",
        "answer": "An ex-parte decree is a court decision made in the absence of one party who fails to appear despite notices."
    },
    {
        "question": "Can an ex-parte divorce decree be challenged?",
        "answer": "Yes, the absent party can file an application for setting aside the decree by providing valid reasons for their absence."
    },
    {
        "question": "What is the status of a marriage with a non-Muslim?",
        "answer": "A Muslim man may marry a woman from Ahl-e-Kitab (People of the Book), but a Muslim woman cannot marry a non-Muslim unless he converts to Islam."
    },
    {
        "question": "Is adoption recognized under Islamic law?",
        "answer": "Islam permits guardianship (kafalah) but does not allow adoption that changes lineage or inheritance rights."
    },
    {
        "question": "Can adopted children inherit from adoptive parents?",
        "answer": "No, adopted children do not automatically inherit under Islamic law unless a will (up to one-third of the estate) is made."
    },
    {
        "question": "Can a woman include a condition for divorce delegation in the Nikahnama?",
        "answer": "Yes, a wife may include a clause for delegated divorce (Talaq-e-Tafweez) with the husband’s consent."
    },
    {
        "question": "Is court marriage legally valid in Pakistan?",
        "answer": "Yes, court marriages are legally valid in Pakistan if all Islamic and legal requirements are fulfilled."
    },
    {
        "question": "Can parental consent be bypassed in a court marriage?",
        "answer": "Yes, if the individuals are adults and mentally sound, they can marry without parental consent."
    },
    {
        "question": "What is a marriage certificate?",
        "answer": "A marriage certificate is an official document issued by the Union Council verifying the legal registration of a marriage."
    },
    {
        "question": "Can talaq be revoked after the iddah period?",
        "answer": "No, once the iddah period ends after a revocable talaq, the divorce becomes final and remarriage requires a new nikah."
    },
    {
        "question": "What is the process to remarry a divorced spouse?",
        "answer": "If the divorce was revocable, a new nikah is needed after iddah. After three talaqs, halala is required before remarriage."
    },
    {
        "question": "Is mutual divorce (Mubarat) allowed in Pakistan?",
        "answer": "Yes, both spouses can agree to dissolve the marriage through Mubarat, and it must be registered legally."
    },
    {
        "question": "What is Mubarat?",
        "answer": "Mubarat is a form of divorce where both husband and wife mutually agree to separate."
    },
    {
        "question": "Can a marriage be annulled?",
        "answer": "Yes, a marriage can be annulled by the court if it was conducted under coercion, fraud, or other legal defects."
    },
    {
        "question": "Does the wife have inheritance rights if husband dies without divorce?",
        "answer": "Yes, if the husband dies without legally divorcing the wife, she inherits according to Islamic laws."
    },
    {
        "question": "What happens to property owned jointly after divorce?",
        "answer": "Jointly owned property may be divided through mutual agreement or a court order based on contributions and ownership records."
    },
    {
        "question": "What is the inheritance share of a Muslim wife?",
        "answer": "A Muslim wife inherits one-fourth of her husband’s estate if there are no children, and one-eighth if there are children."
    },
    {
        "question": "Can marriage be dissolved if the husband is missing?",
        "answer": "Yes, if the husband is missing for four years and cannot be located, the wife can seek dissolution through the court."
    }
]
qa_pairs = [
    {
        "question": "What is the legal age of marriage for a Muslim girl in Pakistan?",
        "answer": "The legal age of marriage for a Muslim girl in Pakistan is 16 years under the Child Marriage Restraint Act. However, there are proposals to raise it to 18 years."
    },
    {
        "question": "Can a girl marry without her wali (guardian) in Pakistan?",
        "answer": "Yes, under Hanafi jurisprudence and Pakistani law, an adult and sane girl can contract marriage without her wali."
    },
    {
        "question": "Is registration of nikah compulsory in Pakistan?",
        "answer": "Yes, under the Muslim Family Laws Ordinance 1961, registration of nikah is mandatory for it to be legally recognized."
    },
    {
        "question": "What is a Nikahnama?",
        "answer": "Nikahnama is the official marriage contract form in Pakistan, documenting the marriage and conditions agreed by both parties."
    },
    {
        "question": "Can conditions be included in the Nikahnama?",
        "answer": "Yes, any lawful conditions mutually agreed upon by the bride and groom can be included in the Nikahnama."
    },
    {
        "question": "Is a verbal nikah valid in Islam?",
        "answer": "Yes, a verbal nikah is valid in Islam if all requirements such as offer, acceptance, witnesses, and mahr are fulfilled."
    },
    {
        "question": "Is consent of the bride necessary in a nikah?",
        "answer": "Yes, a valid nikah cannot be performed without the free and informed consent of the bride."
    },
    {
        "question": "What is Mahr (Dower)?",
        "answer": "Mahr is a mandatory payment by the husband to the wife at the time of marriage, as an Islamic obligation."
    },
    {
        "question": "Can the wife waive her mahr?",
        "answer": "Yes, a wife can waive her mahr after marriage, provided the decision is made voluntarily and without coercion."
    },
    {
        "question": "What are the types of mahr?",
        "answer": "Mahr can be prompt (mu'ajjal) or deferred (mu’akhkhar), meaning it is either paid immediately or at a later agreed time."
    },
    {
        "question": "Can a man contract a second marriage without the first wife's permission?",
        "answer": "No, under Pakistani law, a man must obtain written permission from the Arbitration Council and the first wife before contracting a second marriage."
    },
    {
        "question": "What is the punishment for contracting a second marriage without permission?",
        "answer": "The husband may face up to one year of imprisonment or a fine up to Rs. 500,000, or both, for violating the law."
    },
    {
        "question": "Is muta (temporary marriage) recognized in Pakistan?",
        "answer": "No, muta is not legally recognized in Pakistan, which follows the Sunni Hanafi school of thought."
    },
    {
        "question": "Can a Christian woman convert to Islam and marry a Muslim man in Pakistan?",
        "answer": "Yes, if the conversion is genuine and not solely for the purpose of marriage, she may marry a Muslim man."
    },
    {
        "question": "Is registration of second marriage compulsory?",
        "answer": "Yes, just like the first marriage, all subsequent marriages must also be registered legally."
    },
    {
        "question": "What is Talaq?",
        "answer": "Talaq is the Islamic term for divorce initiated by the husband. It can be oral or written, but legal procedures must be followed for it to be effective in Pakistan."
    },
    {
        "question": "What is the procedure of talaq in Pakistan?",
        "answer": "The husband must send a written notice to the Union Council, which initiates a 90-day reconciliation period. If reconciliation fails, divorce becomes effective after this period."
    },
    {
        "question": "Is oral talaq valid in Pakistan?",
        "answer": "Religiously it may be valid, but legally it is not recognized unless a written notice is given to the Union Council."
    },
    {
        "question": "What is the iddah period after talaq?",
        "answer": "The iddah period is three menstrual cycles, or in the case of a pregnant woman, until childbirth."
    },
    {
        "question": "Can a woman initiate divorce in Pakistan?",
        "answer": "Yes, a woman can seek divorce through Khula, judicial divorce, or if delegated talaq (Talaq-e-Tafweez) is provided in the Nikahnama."
    },
    {
        "question": "What is Khula?",
        "answer": "Khula is a woman’s right to seek divorce by returning her mahr or offering compensation, usually through the Family Court."
    },
    {
        "question": "What is delegated talaq (Talaq-e-Tafweez)?",
        "answer": "It is the right given to the wife in the Nikahnama to pronounce talaq on behalf of herself, if agreed upon by the husband at marriage."
    },
    {
        "question": "What happens if a man gives triple talaq at once?",
        "answer": "Triple talaq is discouraged and counts as one revocable talaq under Pakistani legal interpretation, but legal procedures must still be followed."
    },
    {
        "question": "Can a woman remarry the same man after talaq?",
        "answer": "Yes, if talaq was revocable or if it was a single talaq. After triple talaq, remarriage requires halala (intervening marriage with another man)."
    },
    {
        "question": "What is halala?",
        "answer": "Halala is the requirement that after triple talaq, the woman must marry and be divorced by another man before she can remarry her former husband."
    },
    {
        "question": "What happens if a husband dies during the wife’s iddah after talaq?",
        "answer": "If the husband dies during the iddah period following a revocable talaq, the wife is entitled to inheritance as a widow."
    },
    {
        "question": "What is the iddah period after the death of a husband?",
        "answer": "The iddah period after a husband's death is four months and ten days, or until delivery if the wife is pregnant."
    },
    {
        "question": "Can a woman marry during her iddah period?",
        "answer": "No, a woman cannot marry another man until her iddah period is completed."
    },
    {
        "question": "What is the punishment for child marriage in Pakistan?",
        "answer": "Child marriage is punishable with imprisonment up to three years and a fine under the Child Marriage Restraint Act."
    },
    {
        "question": "Can both parties add conditions in the Nikahnama?",
        "answer": "Yes, both the bride and groom can mutually agree to add lawful conditions in the Nikahnama."
    },
    {
        "question": "What is the role of witnesses in nikah?",
        "answer": "Two adult Muslim witnesses are required for the validity of a nikah under Islamic and Pakistani law."
    },
    {
        "question": "What if the Nikahnama is lost or not available?",
        "answer": "A duplicate can be obtained from the relevant Union Council or Nikah registrar’s office where the marriage was registered."
    },
    {
        "question": "What is an ex-parte decree in family law?",
        "answer": "An ex-parte decree is a court decision made in the absence of one party who fails to appear despite notices."
    },
    {
        "question": "Can an ex-parte divorce decree be challenged?",
        "answer": "Yes, the absent party can file an application for setting aside the decree by providing valid reasons for their absence."
    },
    {
        "question": "What is the status of a marriage with a non-Muslim?",
        "answer": "A Muslim man may marry a woman from Ahl-e-Kitab (People of the Book), but a Muslim woman cannot marry a non-Muslim unless he converts to Islam."
    },
    {
        "question": "Is adoption recognized under Islamic law?",
        "answer": "Islam permits guardianship (kafalah) but does not allow adoption that changes lineage or inheritance rights."
    },
    {
        "question": "Can adopted children inherit from adoptive parents?",
        "answer": "No, adopted children do not automatically inherit under Islamic law unless a will (up to one-third of the estate) is made."
    },
    {
        "question": "Can a woman include a condition for divorce delegation in the Nikahnama?",
        "answer": "Yes, a wife may include a clause for delegated divorce (Talaq-e-Tafweez) with the husband’s consent."
    },
    {
        "question": "Is court marriage legally valid in Pakistan?",
        "answer": "Yes, court marriages are legally valid in Pakistan if all Islamic and legal requirements are fulfilled."
    },
    {
        "question": "Can parental consent be bypassed in a court marriage?",
        "answer": "Yes, if the individuals are adults and mentally sound, they can marry without parental consent."
    },
    {
        "question": "What is a marriage certificate?",
        "answer": "A marriage certificate is an official document issued by the Union Council verifying the legal registration of a marriage."
    },
    {
        "question": "Can talaq be revoked after the iddah period?",
        "answer": "No, once the iddah period ends after a revocable talaq, the divorce becomes final and remarriage requires a new nikah."
    },
    {
        "question": "What is the process to remarry a divorced spouse?",
        "answer": "If the divorce was revocable, a new nikah is needed after iddah. After three talaqs, halala is required before remarriage."
    },
    {
        "question": "Is mutual divorce (Mubarat) allowed in Pakistan?",
        "answer": "Yes, both spouses can agree to dissolve the marriage through Mubarat, and it must be registered legally."
    },
    {
        "question": "What is Mubarat?",
        "answer": "Mubarat is a form of divorce where both husband and wife mutually agree to separate."
    },
    {
        "question": "Can a marriage be annulled?",
        "answer": "Yes, a marriage can be annulled by the court if it was conducted under coercion, fraud, or other legal defects."
    },
    {
        "question": "Does the wife have inheritance rights if husband dies without divorce?",
        "answer": "Yes, if the husband dies without legally divorcing the wife, she inherits according to Islamic laws."
    },
    {
        "question": "What happens to property owned jointly after divorce?",
        "answer": "Jointly owned property may be divided through mutual agreement or a court order based on contributions and ownership records."
    },
    {
        "question": "What is the inheritance share of a Muslim wife?",
        "answer": "A Muslim wife inherits one-fourth of her husband’s estate if there are no children, and one-eighth if there are children."
    },
    {
        "question": "Can marriage be dissolved if the husband is missing?",
        "answer": "Yes, if the husband is missing for four years and cannot be located, the wife can seek dissolution through the court."
    }
]
qa_pairs += [
    {
        "question": "What is the legal age of marriage for a Muslim girl in Pakistan?",
        "answer": "The legal age of marriage for a Muslim girl in Pakistan is 16 years under the Child Marriage Restraint Act. However, there are proposals to raise it to 18 years."
    },
    {
        "question": "Can a girl marry without her wali (guardian) in Pakistan?",
        "answer": "Yes, under Hanafi jurisprudence and Pakistani law, an adult and sane girl can contract marriage without her wali."
    },
    {
        "question": "Is registration of nikah compulsory in Pakistan?",
        "answer": "Yes, under the Muslim Family Laws Ordinance 1961, registration of nikah is mandatory for it to be legally recognized."
    },
    {
        "question": "What is a Nikahnama?",
        "answer": "Nikahnama is the official marriage contract form in Pakistan, documenting the marriage and conditions agreed by both parties."
    },
    {
        "question": "Can conditions be included in the Nikahnama?",
        "answer": "Yes, any lawful conditions mutually agreed upon by the bride and groom can be included in the Nikahnama."
    },
    {
        "question": "Is a verbal nikah valid in Islam?",
        "answer": "Yes, a verbal nikah is valid in Islam if all requirements such as offer, acceptance, witnesses, and mahr are fulfilled."
    },
    {
        "question": "Is consent of the bride necessary in a nikah?",
        "answer": "Yes, a valid nikah cannot be performed without the free and informed consent of the bride."
    },
    {
        "question": "What is Mahr (Dower)?",
        "answer": "Mahr is a mandatory payment by the husband to the wife at the time of marriage, as an Islamic obligation."
    },
    {
        "question": "Can the wife waive her mahr?",
        "answer": "Yes, a wife can waive her mahr after marriage, provided the decision is made voluntarily and without coercion."
    },
    {
        "question": "What are the types of mahr?",
        "answer": "Mahr can be prompt (mu'ajjal) or deferred (mu’akhkhar), meaning it is either paid immediately or at a later agreed time."
    },
    {
        "question": "Can a man contract a second marriage without the first wife's permission?",
        "answer": "No, under Pakistani law, a man must obtain written permission from the Arbitration Council and the first wife before contracting a second marriage."
    },
    {
        "question": "What is the punishment for contracting a second marriage without permission?",
        "answer": "The husband may face up to one year of imprisonment or a fine up to Rs. 500,000, or both, for violating the law."
    },
    {
        "question": "Is muta (temporary marriage) recognized in Pakistan?",
        "answer": "No, muta is not legally recognized in Pakistan, which follows the Sunni Hanafi school of thought."
    },
    {
        "question": "Can a Christian woman convert to Islam and marry a Muslim man in Pakistan?",
        "answer": "Yes, if the conversion is genuine and not solely for the purpose of marriage, she may marry a Muslim man."
    },
    {
        "question": "Is registration of second marriage compulsory?",
        "answer": "Yes, just like the first marriage, all subsequent marriages must also be registered legally."
    },
    {
        "question": "What is Talaq?",
        "answer": "Talaq is the Islamic term for divorce initiated by the husband. It can be oral or written, but legal procedures must be followed for it to be effective in Pakistan."
    },
    {
        "question": "What is the procedure of talaq in Pakistan?",
        "answer": "The husband must send a written notice to the Union Council, which initiates a 90-day reconciliation period. If reconciliation fails, divorce becomes effective after this period."
    },
    {
        "question": "Is oral talaq valid in Pakistan?",
        "answer": "Religiously it may be valid, but legally it is not recognized unless a written notice is given to the Union Council."
    },
    {
        "question": "What is the iddah period after talaq?",
        "answer": "The iddah period is three menstrual cycles, or in the case of a pregnant woman, until childbirth."
    },
    {
        "question": "Can a woman initiate divorce in Pakistan?",
        "answer": "Yes, a woman can seek divorce through Khula, judicial divorce, or if delegated talaq (Talaq-e-Tafweez) is provided in the Nikahnama."
    },
    {
        "question": "What is Khula?",
        "answer": "Khula is a woman’s right to seek divorce by returning her mahr or offering compensation, usually through the Family Court."
    },
    {
        "question": "What is delegated talaq (Talaq-e-Tafweez)?",
        "answer": "It is the right given to the wife in the Nikahnama to pronounce talaq on behalf of herself, if agreed upon by the husband at marriage."
    },
    {
        "question": "What happens if a man gives triple talaq at once?",
        "answer": "Triple talaq is discouraged and counts as one revocable talaq under Pakistani legal interpretation, but legal procedures must still be followed."
    },
    {
        "question": "Can a woman remarry the same man after talaq?",
        "answer": "Yes, if talaq was revocable or if it was a single talaq. After triple talaq, remarriage requires halala (intervening marriage with another man)."
    },
    {
        "question": "What is halala?",
        "answer": "Halala is the requirement that after triple talaq, the woman must marry and be divorced by another man before she can remarry her former husband."
    }
]
qa_pairs += [
    {
        "question": "Is court approval necessary for khula in Pakistan?",
        "answer": "Yes, khula must be granted by a Family Court in Pakistan after the wife files a suit and shows valid reasons."
    },
    {
        "question": "Can a woman file for judicial divorce in Pakistan?",
        "answer": "Yes, under the Dissolution of Muslim Marriages Act 1939, a woman can seek judicial divorce on various grounds like cruelty, desertion, or impotence."
    },
    {
        "question": "What are valid grounds for judicial divorce?",
        "answer": "Grounds include husband's cruelty, desertion for two years, failure to maintain for two years, imprisonment, impotence, or failure to perform marital obligations."
    },
    {
        "question": "Is maintenance obligatory after divorce?",
        "answer": "Yes, a husband is obligated to provide maintenance during the iddah period. After that, it depends on the circumstances and court orders."
    },
    {
        "question": "What is the law on child custody after divorce?",
        "answer": "Custody is usually granted to the mother, especially for young children, but the court decides based on the best interest of the child."
    },
    {
        "question": "Can the father claim custody of children?",
        "answer": "Yes, the father can apply for custody, particularly after the child reaches a certain age or if the mother is deemed unfit."
    },
    {
        "question": "What is guardianship (wilayah) under Islamic law?",
        "answer": "Guardianship refers to the legal responsibility over a child's person or property. Usually, the father retains guardianship even if the mother has custody."
    },
    {
        "question": "What is the difference between custody and guardianship?",
        "answer": "Custody is the right to physical care of the child, while guardianship refers to legal decision-making authority and financial responsibilities."
    },
    {
        "question": "Can a mother remarry and still retain custody?",
        "answer": "Courts may transfer custody to the father or another guardian if the mother remarries, depending on the child's welfare."
    },
    {
        "question": "What is maintenance (nafqah) under Islamic law?",
        "answer": "Maintenance refers to the husband’s obligation to provide food, clothing, shelter, and medical care to his wife and children."
    },
    {
        "question": "Can a woman claim past maintenance?",
        "answer": "Yes, she may claim arrears of maintenance for a reasonable past period, subject to court discretion."
    },
    {
        "question": "Is the father's consent required for a child's travel abroad after divorce?",
        "answer": "Yes, generally the non-custodial parent's consent is required for international travel, especially when there’s a court decree involved."
    },
    {
        "question": "Can the mother apply for child maintenance separately?",
        "answer": "Yes, the mother can file a separate suit in Family Court for child maintenance from the father."
    },
    {
        "question": "What is the role of Union Councils in family disputes?",
        "answer": "Union Councils handle reconciliation in talaq cases, issue marriage/divorce certificates, and supervise arbitration proceedings."
    },
    {
        "question": "Are family court proceedings public in Pakistan?",
        "answer": "No, Family Court proceedings are generally held in-camera to protect privacy and sensitive issues."
    },
    {
        "question": "Can a husband revoke talaq during iddah?",
        "answer": "Yes, if it is the first or second talaq, the husband may revoke it during the iddah period through verbal or written declaration or resuming marital relations."
    },
    {
        "question": "Can talaq be given through SMS or WhatsApp?",
        "answer": "While some courts may accept electronic communication as evidence, legal procedures such as notice to the Union Council are still mandatory."
    },
    {
        "question": "Is forced marriage allowed under Islamic law?",
        "answer": "No, Islam strictly prohibits forced marriages. Consent of both parties is a fundamental requirement."
    },
    {
        "question": "What is the punishment for forced marriage in Pakistan?",
        "answer": "Forced marriage is a criminal offense under the Prevention of Anti-Women Practices Act, punishable with imprisonment and fine."
    },
    {
        "question": "Can a minor girl be married under Islamic law?",
        "answer": "Marriage of a minor is not valid unless she reaches puberty and gives free consent. Pakistani law also sets a minimum legal age."
    },
    {
        "question": "What is the role of arbitration councils in second marriages?",
        "answer": "Arbitration Councils evaluate applications for second marriages, ensuring that valid reasons and first wife’s consent are provided."
    },
    {
        "question": "What documents are required for Nikah registration?",
        "answer": "Documents usually include CNICs of bride, groom, and witnesses, photographs, and filled Nikahnama with signatures."
    },
    {
        "question": "Can a woman be a nikah registrar in Pakistan?",
        "answer": "Currently, Nikah registrars are typically male, but legally there is no restriction preventing women from being appointed."
    },
    {
        "question": "What is the legal status of an unregistered nikah?",
        "answer": "An unregistered nikah may still be valid religiously but is not recognized legally without proper documentation."
    },
    {
        "question": "Can a divorced woman claim inheritance from her former husband?",
        "answer": "No, a divorced woman is not entitled to inherit from her former husband unless he passes away during her iddah."
    }
]
qa_pairs += [
    {
        "question": "What happens if a husband dies during the wife’s iddah after talaq?",
        "answer": "If the husband dies during the iddah period following a revocable talaq, the wife is entitled to inheritance as a widow."
    },
    {
        "question": "What is the iddah period after the death of a husband?",
        "answer": "The iddah period after a husband's death is four months and ten days, or until delivery if the wife is pregnant."
    },
    {
        "question": "Can a woman marry during her iddah period?",
        "answer": "No, a woman cannot marry another man until her iddah period is completed."
    },
    {
        "question": "What is the punishment for child marriage in Pakistan?",
        "answer": "Child marriage is punishable with imprisonment up to three years and a fine under the Child Marriage Restraint Act."
    },
    {
        "question": "Can both parties add conditions in the Nikahnama?",
        "answer": "Yes, both the bride and groom can mutually agree to add lawful conditions in the Nikahnama."
    },
    {
        "question": "What is the role of witnesses in nikah?",
        "answer": "Two adult Muslim witnesses are required for the validity of a nikah under Islamic and Pakistani law."
    },
    {
        "question": "What if the Nikahnama is lost or not available?",
        "answer": "A duplicate can be obtained from the relevant Union Council or Nikah registrar’s office where the marriage was registered."
    },
    {
        "question": "What is an ex-parte decree in family law?",
        "answer": "An ex-parte decree is a court decision made in the absence of one party who fails to appear despite notices."
    },
    {
        "question": "Can an ex-parte divorce decree be challenged?",
        "answer": "Yes, the absent party can file an application for setting aside the decree by providing valid reasons for their absence."
    },
    {
        "question": "What is the status of a marriage with a non-Muslim?",
        "answer": "A Muslim man may marry a woman from Ahl-e-Kitab (People of the Book), but a Muslim woman cannot marry a non-Muslim unless he converts to Islam."
    },
    {
        "question": "Is adoption recognized under Islamic law?",
        "answer": "Islam permits guardianship (kafalah) but does not allow adoption that changes lineage or inheritance rights."
    },
    {
        "question": "Can adopted children inherit from adoptive parents?",
        "answer": "No, adopted children do not automatically inherit under Islamic law unless a will (up to one-third of the estate) is made."
    },
    {
        "question": "Can a woman include a condition for divorce delegation in the Nikahnama?",
        "answer": "Yes, a wife may include a clause for delegated divorce (Talaq-e-Tafweez) with the husband’s consent."
    },
    {
        "question": "Is court marriage legally valid in Pakistan?",
        "answer": "Yes, court marriages are legally valid in Pakistan if all Islamic and legal requirements are fulfilled."
    },
    {
        "question": "Can parental consent be bypassed in a court marriage?",
        "answer": "Yes, if the individuals are adults and mentally sound, they can marry without parental consent."
    },
    {
        "question": "What is a marriage certificate?",
        "answer": "A marriage certificate is an official document issued by the Union Council verifying the legal registration of a marriage."
    },
    {
        "question": "Can talaq be revoked after the iddah period?",
        "answer": "No, once the iddah period ends after a revocable talaq, the divorce becomes final and remarriage requires a new nikah."
    },
    {
        "question": "What is the process to remarry a divorced spouse?",
        "answer": "If the divorce was revocable, a new nikah is needed after iddah. After three talaqs, halala is required before remarriage."
    },
    {
        "question": "Is mutual divorce (Mubarat) allowed in Pakistan?",
        "answer": "Yes, both spouses can agree to dissolve the marriage through Mubarat, and it must be registered legally."
    },
    {
        "question": "What is Mubarat?",
        "answer": "Mubarat is a form of divorce where both husband and wife mutually agree to separate."
    },
    {
        "question": "Can a marriage be annulled?",
        "answer": "Yes, a marriage can be annulled by the court if it was conducted under coercion, fraud, or other legal defects."
    },
    {
        "question": "Does the wife have inheritance rights if husband dies without divorce?",
        "answer": "Yes, if the husband dies without legally divorcing the wife, she inherits according to Islamic laws."
    },
    {
        "question": "What happens to property owned jointly after divorce?",
        "answer": "Jointly owned property may be divided through mutual agreement or a court order based on contributions and ownership records."
    },
    {
        "question": "What is the inheritance share of a Muslim wife?",
        "answer": "A Muslim wife inherits one-fourth of her husband’s estate if there are no children, and one-eighth if there are children."
    },
    {
        "question": "Can marriage be dissolved if the husband is missing?",
        "answer": "Yes, if the husband is missing for four years and cannot be located, the wife can seek dissolution through the court."
    }
]
qa_pairs += [
    {
        "question": "What is the legal age of marriage for a Muslim girl in Pakistan?",
        "answer": "The legal age of marriage for a Muslim girl in Pakistan is 16 years under the Child Marriage Restraint Act. However, there are proposals to raise it to 18 years."
    },
    {
        "question": "Can a girl marry without her wali (guardian) in Pakistan?",
        "answer": "Yes, under Hanafi jurisprudence and Pakistani law, an adult and sane girl can contract marriage without her wali."
    },
    {
        "question": "Is registration of nikah compulsory in Pakistan?",
        "answer": "Yes, under the Muslim Family Laws Ordinance 1961, registration of nikah is mandatory for it to be legally recognized."
    },
    {
        "question": "What is a Nikahnama?",
        "answer": "Nikahnama is the official marriage contract form in Pakistan, documenting the marriage and conditions agreed by both parties."
    },
    {
        "question": "Can conditions be included in the Nikahnama?",
        "answer": "Yes, any lawful conditions mutually agreed upon by the bride and groom can be included in the Nikahnama."
    },
    {
        "question": "Is a verbal nikah valid in Islam?",
        "answer": "Yes, a verbal nikah is valid in Islam if all requirements such as offer, acceptance, witnesses, and mahr are fulfilled."
    },
    {
        "question": "Is consent of the bride necessary in a nikah?",
        "answer": "Yes, a valid nikah cannot be performed without the free and informed consent of the bride."
    },
    {
        "question": "What is Mahr (Dower)?",
        "answer": "Mahr is a mandatory payment by the husband to the wife at the time of marriage, as an Islamic obligation."
    },
    {
        "question": "Can the wife waive her mahr?",
        "answer": "Yes, a wife can waive her mahr after marriage, provided the decision is made voluntarily and without coercion."
    },
    {
        "question": "What are the types of mahr?",
        "answer": "Mahr can be prompt (mu'ajjal) or deferred (mu’akhkhar), meaning it is either paid immediately or at a later agreed time."
    },
    {
        "question": "Can a man contract a second marriage without the first wife's permission?",
        "answer": "No, under Pakistani law, a man must obtain written permission from the Arbitration Council and the first wife before contracting a second marriage."
    },
    {
        "question": "What is the punishment for contracting a second marriage without permission?",
        "answer": "The husband may face up to one year of imprisonment or a fine up to Rs. 500,000, or both, for violating the law."
    },
    {
        "question": "Is muta (temporary marriage) recognized in Pakistan?",
        "answer": "No, muta is not legally recognized in Pakistan, which follows the Sunni Hanafi school of thought."
    },
    {
        "question": "Can a Christian woman convert to Islam and marry a Muslim man in Pakistan?",
        "answer": "Yes, if the conversion is genuine and not solely for the purpose of marriage, she may marry a Muslim man."
    },
    {
        "question": "Is registration of second marriage compulsory?",
        "answer": "Yes, just like the first marriage, all subsequent marriages must also be registered legally."
    },
    {
        "question": "What is Talaq?",
        "answer": "Talaq is the Islamic term for divorce initiated by the husband. It can be oral or written, but legal procedures must be followed for it to be effective in Pakistan."
    },
    {
        "question": "What is the procedure of talaq in Pakistan?",
        "answer": "The husband must send a written notice to the Union Council, which initiates a 90-day reconciliation period. If reconciliation fails, divorce becomes effective after this period."
    },
    {
        "question": "Is oral talaq valid in Pakistan?",
        "answer": "Religiously it may be valid, but legally it is not recognized unless a written notice is given to the Union Council."
    },
    {
        "question": "What is the iddah period after talaq?",
        "answer": "The iddah period is three menstrual cycles, or in the case of a pregnant woman, until childbirth."
    },
    {
        "question": "Can a woman initiate divorce in Pakistan?",
        "answer": "Yes, a woman can seek divorce through Khula, judicial divorce, or if delegated talaq (Talaq-e-Tafweez) is provided in the Nikahnama."
    },
    {
        "question": "What is Khula?",
        "answer": "Khula is a woman’s right to seek divorce by returning her mahr or offering compensation, usually through the Family Court."
    },
    {
        "question": "What is delegated talaq (Talaq-e-Tafweez)?",
        "answer": "It is the right given to the wife in the Nikahnama to pronounce talaq on behalf of herself, if agreed upon by the husband at marriage."
    },
    {
        "question": "What happens if a man gives triple talaq at once?",
        "answer": "Triple talaq is discouraged and counts as one revocable talaq under Pakistani legal interpretation, but legal procedures must still be followed."
    },
    {
        "question": "Can a woman remarry the same man after talaq?",
        "answer": "Yes, if talaq was revocable or if it was a single talaq. After triple talaq, remarriage requires halala (intervening marriage with another man)."
    },
    {
        "question": "What is halala?",
        "answer": "Halala is the requirement that after triple talaq, the woman must marry and be divorced by another man before she can remarry her former husband."
    },
    {
        "question": "What happens if a husband dies during the wife’s iddah after talaq?",
        "answer": "If the husband dies during the iddah period following a revocable talaq, the wife is entitled to inheritance as a widow."
    },
    {
        "question": "What is the iddah period after the death of a husband?",
        "answer": "The iddah period after a husband's death is four months and ten days, or until delivery if the wife is pregnant."
    },
    {
        "question": "Can a woman marry during her iddah period?",
        "answer": "No, a woman cannot marry another man until her iddah period is completed."
    },
    {
        "question": "What is the punishment for child marriage in Pakistan?",
        "answer": "Child marriage is punishable with imprisonment up to three years and a fine under the Child Marriage Restraint Act."
    },
    {
        "question": "Can both parties add conditions in the Nikahnama?",
        "answer": "Yes, both the bride and groom can mutually agree to add lawful conditions in the Nikahnama."
    },
    {
        "question": "What is the role of witnesses in nikah?",
        "answer": "Two adult Muslim witnesses are required for the validity of a nikah under Islamic and Pakistani law."
    },
    {
        "question": "What if the Nikahnama is lost or not available?",
        "answer": "A duplicate can be obtained from the relevant Union Council or Nikah registrar’s office where the marriage was registered."
    },
    {
        "question": "What is an ex-parte decree in family law?",
        "answer": "An ex-parte decree is a court decision made in the absence of one party who fails to appear despite notices."
    },
    {
        "question": "Can an ex-parte divorce decree be challenged?",
        "answer": "Yes, the absent party can file an application for setting aside the decree by providing valid reasons for their absence."
    },
    {
        "question": "What is the status of a marriage with a non-Muslim?",
        "answer": "A Muslim man may marry a woman from Ahl-e-Kitab (People of the Book), but a Muslim woman cannot marry a non-Muslim unless he converts to Islam."
    },
    {
        "question": "Is adoption recognized under Islamic law?",
        "answer": "Islam permits guardianship (kafalah) but does not allow adoption that changes lineage or inheritance rights."
    },
    {
        "question": "Can adopted children inherit from adoptive parents?",
        "answer": "No, adopted children do not automatically inherit under Islamic law unless a will (up to one-third of the estate) is made."
    },
    {
        "question": "Can a woman include a condition for divorce delegation in the Nikahnama?",
        "answer": "Yes, a wife may include a clause for delegated divorce (Talaq-e-Tafweez) with the husband’s consent."
    },
    {
        "question": "Is court marriage legally valid in Pakistan?",
        "answer": "Yes, court marriages are legally valid in Pakistan if all Islamic and legal requirements are fulfilled."
    },
    {
        "question": "Can parental consent be bypassed in a court marriage?",
        "answer": "Yes, if the individuals are adults and mentally sound, they can marry without parental consent."
    },
    {
        "question": "What is a marriage certificate?",
        "answer": "A marriage certificate is an official document issued by the Union Council verifying the legal registration of a marriage."
    },
    {
        "question": "Can talaq be revoked after the iddah period?",
        "answer": "No, once the iddah period ends after a revocable talaq, the divorce becomes final and remarriage requires a new nikah."
    },
    {
        "question": "What is the process to remarry a divorced spouse?",
        "answer": "If the divorce was revocable, a new nikah is needed after iddah. After three talaqs, halala is required before remarriage."
    },
    {
        "question": "Is mutual divorce (Mubarat) allowed in Pakistan?",
        "answer": "Yes, both spouses can agree to dissolve the marriage through Mubarat, and it must be registered legally."
    },
    {
        "question": "What is Mubarat?",
        "answer": "Mubarat is a form of divorce where both husband and wife mutually agree to separate."
    },
    {
        "question": "Can a marriage be annulled?",
        "answer": "Yes, a marriage can be annulled by the court if it was conducted under coercion, fraud, or other legal defects."
    },
    {
        "question": "Does the wife have inheritance rights if husband dies without divorce?",
        "answer": "Yes, if the husband dies without legally divorcing the wife, she inherits according to Islamic laws."
    },
    {
        "question": "What happens to property owned jointly after divorce?",
        "answer": "Jointly owned property may be divided through mutual agreement or a court order based on contributions and ownership records."
    },
    {
        "question": "What is the inheritance share of a Muslim wife?",
        "answer": "A Muslim wife inherits one-fourth of her husband’s estate if there are no children, and one-eighth if there are children."
    },
    {
        "question": "Can marriage be dissolved if the husband is missing?",
        "answer": "Yes, if the husband is missing for four years and cannot be located, the wife can seek dissolution through the court."
    }
]


# Step 4: Save to JSONL
with open("muslim_family_law_qa.jsonl", "w") as f:
    for item in qa_pairs:
        f.write(json.dumps(item) + "\n")

# Step 5: Load and prepare dataset
dataset = load_dataset("json", data_files="muslim_family_law_qa.jsonl")["train"]

def format_example(example):
    return {
        "text": f"### Instruction:\n{example['question']}\n\n### Response:\n{example['answer']}"
    }

dataset = dataset.map(format_example)

# Step 6: Load TinyLlama model and tokenizer (4-bit quantized)
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # 🔧 Fix padding issue

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(load_in_4bit=True)
)

# Step 7: PEFT + LoRA config
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Step 8: Tokenize dataset
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)

# ⚙️ Step 9: Training config
training_args = TrainingArguments(
    output_dir="./tinyllama-legal-bot",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# 🚀 Step 10: Train!
trainer.train()


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recom

Step,Training Loss
10,2.786600
20,2.650600
30,2.560400
40,2.505500
50,2.252700
60,2.121600
70,2.111300
80,1.906000
90,1.815700
100,1.728800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=264, training_loss=1.7545384565989177, metrics={'train_runtime': 189.2921, 'train_samples_per_second': 2.773, 'train_steps_per_second': 1.395, 'total_flos': 1670278230835200.0, 'train_loss': 1.7545384565989177, 'epoch': 3.0})

In [6]:
model.save_pretrained("tinyllama_lora_muslim_family_law")
tokenizer.save_pretrained("tinyllama_lora_muslim_family_law")


('tinyllama_lora_muslim_family_law/tokenizer_config.json',
 'tinyllama_lora_muslim_family_law/special_tokens_map.json',
 'tinyllama_lora_muslim_family_law/tokenizer.model',
 'tinyllama_lora_muslim_family_law/added_tokens.json',
 'tinyllama_lora_muslim_family_law/tokenizer.json')

In [7]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("tinyllama_lora_muslim_family_law")

model = PeftModel.from_pretrained(base_model, "tinyllama_lora_muslim_family_law")
model.eval()

# Inference
prompt = "Q: What is the role of witnesses in nikah?\nA:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Q: What is the role of witnesses in nikah?
A: Witnesses are required in nikah. They are not required in marriage contracts.
Q: What is the role of the witnesses in nikah?
A: The witnesses are required to confirm the marriage contract. They are not required to confirm the marriage itself.
Q: What is the role of the witnesses in marriage contracts?
A: The witnesses are required to confirm the marriage contract. They are not required to confirm the marriage itself.

